In [1]:
// #include <stdio.h>
// #include <stdlib.h>
// #define size 40 //equivalent to 2 windows of 20 samples each

// float buf[size]; //initialize buffer
// float steady[(size/2)*5]; //initialize a second buffer to store 5 windows worth of data
// int headIndex = 0; //keeps track of the 1st element's position
// int thres = 40; //define threshold in watts
// int flag=0; // if flag==0 event detection is ongoing. If flag==1 a 2nd buffer is filled with next 100 samples
// int i=0; //number of elements in steady
// int counter=0; // number of elements in buf

// int rollbuf(float tail)
// {
//     buf[headIndex] = tail; //new entry is stored at headIndex position
//     headIndex = (headIndex + 1) % size; 
//     counter++;
//     return 0;
// }


// void simul(float raw)
// {
//     if (flag==0)
//     {
        
//         float s1 = 0.0, s2 = 0.0, avg1,avg2;
        
//         rollbuf(raw);
//         if (counter >= size) //if buffer is full, calculate averages
//         {
//             int j;
//             for (j=headIndex;j<(size/2)+headIndex;j++)
//             {
//                 s1 += buf[j%size]; //compute sum of 1st window
//                 s2 += buf[(j%size+(size/2))%size]; //compute sum of 2nd window
//             }
            
//             //compute corresponding averages
//             avg1 = s1/(size/2); 
//             avg2 = s2/(size/2);

//             if ((avg1-avg2)>thres || (avg2-avg1)>thres) //event detected
//             {
//                 //printf("Event detected \n");
//                 flag=1;
//                 i=0;
//             } 
//         }
//     }
    
//     else
//     {
//         // if steady buffer is not full, keep storing samples
//         if (i < (sizeof(steady) / sizeof(float))) 
//         {
//             //Store next 5 windows' samples
//             steady[i] = raw;
//             i++;
//         }
//         else
//         {
//             // send data of both buffers to the cloud 
//             flag=0;
//             i=0;
//             counter=0;
            
//         }
//     }

// }
    
// int random_stream(int max_reps)
// {
//     float x = 0;
//     for (int n=0; n<max_reps;n++)
//     {
//         x = rand();
//         simul(x);
//     }
//     return 0;
// }

// int main(int argc, char **argv) {    
    
//     int y;
//     y = random_stream(1000);
    
//     return 0;
// }

In [15]:
#include <stdio.h>
#include <stdlib.h>


#define SIZE 40 //equivalent to 2 windows of 20 samples each
#define HALF_SIZE ( SIZE / 2 )
#define QUART_SIZE ( HALF_SIZE / 2)
#define ROLLED(IDX) ( (IDX < HALF_SIZE ? SIZE : 0) + IDX - HALF_SIZE )
#define CURR 1  // Index of current sum/average
#define PREV 0  // Index Previous sum/average
#define STEADY_SIZE (HALF_SIZE * 5)
#define MAX_REPS 1000
#define RANGE 1000
#define THRES 40.0  //define threshold in watts

#define MIN(A, B) ( A < B ? A : B )
#define ABS(X) ( X < 0 ? -X : X )



void clear_buff(float buf[])
    /* clear the buffer */
{
    for(int i=0; i<SIZE; i++) {
        buf[i] = 0.0;
    }
}
    

void eventoring(float raw, float buf[], float steady[])
/* Monitor events by checking rolling averages of streaming input*/
{
    static float sums[2] = { 0, 0 };
    float avgs[2] = { 0, 0 };
    float rolled;
    static float prev;
    // if flag==0 event detection is ongoing. If flag==1 a 2nd buffer is filled with next 100 samples
    static unsigned int scount=0, flag=0, counter=0;   
    unsigned int index;
    

    index = counter++ % SIZE;

    if (!flag)    
    {
        /* gradually calculate averages*/
        
  
        rolled = buf[ROLLED(index)]; 
    
        /* add the new value, remove the replaced one */
        sums[CURR] += raw - rolled;
        
        if (counter <= SIZE){
            sums[PREV] += rolled - buf[index]; 
        }
        
        
        printf("%5u. buf[%u] was %.3f, becoming %.3f, rolled is buf[%u] = %.3f.\n", counter, index, buf[index], raw, ROLLED(index), rolled);
        
        buf[index] = raw;
        printf("%5u. Input is %.3f. Current sums are %.3f // %.3f. \n", counter, raw, sums[0], sums[1]);
        
        /*Check if buffer is full and modulo HALF_SIZE == 0*/
        if ((counter >= SIZE)&&(!(counter % HALF_SIZE)))
        {
            printf("checking averages...\n");
            for (int j=0; j<2; j++) {
                avgs[j] = sums[j] / HALF_SIZE;
            }
            
            if (ABS(avgs[0]-avgs[1]) > THRES) 
            {
                printf("Event detected. Averages differ by %.3f \n", avgs[0]-avgs[1]);
                for (int i=(index+1)%SIZE; i<((index+1)%SIZE)+QUART_SIZE; i++){
                    prev += buf[i];
//                     printf("summing i= %f\n",prev);
                }
                printf("average of steady state is %f\n",prev/(HALF_SIZE/2));
                clear_buff(buf);
                flag=1;
            }
            
            /*every HALF_SIZE samples shift CURR sum to PREV sum*/
            if (!(counter % HALF_SIZE)){
                sums[PREV] = sums[CURR];
            }
        }

        
    } else {
        // if steady buffer is not full, keep storing samples
        if (scount < STEADY_SIZE) 
        {
            //Store next 5 windows' samples
            steady[scount++] = raw;
        }
        else
        {
            // TODO: send data of both buffers to the cloud 
            flag=0; scount=0; counter=0; sums[0]=0; sums[1]=0; prev=0;
//             clear_buff(buf);
        }
    }

}




int main(int argc, char **argv)
{    
    unsigned int reps = MAX_REPS;
    float buf[SIZE]; // initialize buffer
    float steady[STEADY_SIZE]; //initialize a second buffer to store 5 windows worth of data
    
    clear_buff(buf);

    if (argc > 1) {
        reps = atoi(argv[1]);
    }

    int k;
    for(int j=0; j<reps; j++) {
        eventoring(rand() % RANGE, buf, steady);
//         eventoring(k, buf, steady);
//         k = k+5;
    }
    
    return 0;
}




    1. buf[0] was 0.000, becoming 383.000, rolled is buf[20] = 0.000.
    1. Input is 383.000. Current sums are 0.000 // 383.000. 
    2. buf[1] was 0.000, becoming 886.000, rolled is buf[21] = 0.000.
    2. Input is 886.000. Current sums are 0.000 // 1269.000. 
    3. buf[2] was 0.000, becoming 777.000, rolled is buf[22] = 0.000.
    3. Input is 777.000. Current sums are 0.000 // 2046.000. 
    4. buf[3] was 0.000, becoming 915.000, rolled is buf[23] = 0.000.
    4. Input is 915.000. Current sums are 0.000 // 2961.000. 
    5. buf[4] was 0.000, becoming 793.000, rolled is buf[24] = 0.000.
    5. Input is 793.000. Current sums are 0.000 // 3754.000. 
    6. buf[5] was 0.000, becoming 335.000, rolled is buf[25] = 0.000.
    6. Input is 335.000. Current sums are 0.000 // 4089.000. 
    7. buf[6] was 0.000, becoming 386.000, rolled is buf[26] = 0.000.
    7. Input is 386.000. Current sums are 0.000 // 4475.000. 
    8. buf[7] was 0.000, becoming 492.000, rolled is buf[27] = 0.000.
    8. 

   36. buf[35] was 0.000, becoming 911.000, rolled is buf[15] = 404.000.
   36. Input is 911.000. Current sums are 4721.000 // 11918.000. 
   37. buf[36] was 0.000, becoming 127.000, rolled is buf[16] = 338.000.
   37. Input is 127.000. Current sums are 5059.000 // 11707.000. 
   38. buf[37] was 0.000, becoming 950.000, rolled is buf[17] = 580.000.
   38. Input is 950.000. Current sums are 5639.000 // 12077.000. 
   39. buf[38] was 0.000, becoming 236.000, rolled is buf[18] = 218.000.
   39. Input is 236.000. Current sums are 5857.000 // 12095.000. 
   40. buf[39] was 0.000, becoming 560.000, rolled is buf[19] = 21.000.
   40. Input is 560.000. Current sums are 5878.000 // 12634.000. 
checking averages...
   41. buf[0] was 231.000, becoming 818.000, rolled is buf[20] = 970.000.
   41. Input is 818.000. Current sums are 12634.000 // 12482.000. 
   42. buf[1] was 21.000, becoming 105.000, rolled is buf[21] = 862.000.
   42. Input is 105.000. Current sums are 12634.000 // 11725.000. 
   4